In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2024
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [2]:
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt
import os, warnings, sys, logging
import pandas as pd
import numpy as np
from datetime import date
import cml.data_v1 as cmldata
import pyspark.pandas as ps
import seaborn as sns

In [5]:
USERNAME = os.environ["PROJECT_OWNER"]
DBNAME = "GEOSPATIAL_".format(USERNAME)
STORAGE = "s3a://go01-demo"
CONNECTION_NAME = "go01-aw-dl"
DATE = date.today()

conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

In [4]:
df_from_sql = ps.read_table('{0}.GEOSPATIAL_{1}'.format(DBNAME, USERNAME))
df = df_from_sql.to_pandas()

24/03/16 01:37:02 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
Hive Session ID = 32113003-9e18-4830-9487-cb934d003ee0
24/03/16 01:37:05 WARN HiveMetaStoreClient: Failed to connect to the MetaStore Server...
24/03/16 01:37:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/16 01:37:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [6]:
df['iot_signal_1'] = df['iot_signal_1'].astype("float64")
df['iot_signal_2'] = df['iot_signal_2'].astype("float64")
df['iot_signal_3'] = df['iot_signal_3'].astype("float64")
df['iot_signal_4'] = df['iot_signal_4'].astype("float64")

In [7]:
# Create point geometries
geometry = geopandas.points_from_xy(df.longitude, df.latitude)
geo_df = geopandas.GeoDataFrame(
    df[["id", "device_id", "manufacturer", "event_type", \
        "iot_signal_1", "iot_signal_2", "iot_signal_3", "iot_signal_4"]], geometry=geometry
)

geo_df.head()

,id,device_id,manufacturer,event_type,iot_signal_1,iot_signal_2,iot_signal_3,iot_signal_4,geometry
0,0,0x1000000000005,New World Corp,system malfunction,3.0,3.0,51.0,104.0,POINT (-92.96849 42.29797)
1,1,0x100000000001d,New World Corp,tank below 10%,9.0,9.0,52.0,103.0,POINT (-92.67342 41.75997)
2,2,0x1000000000008,New World Corp,tank below 10%,6.0,7.0,54.0,103.0,POINT (-93.43854 41.90818)
3,3,0x100000000001b,AI Inc.,tank below 10%,2.0,3.0,53.0,105.0,POINT (-92.73906 41.67735)
4,4,0x1000000000014,New World Corp,tank below 10%,10.0,10.0,55.0,102.0,POINT (-93.14474 42.03117)


In [8]:
# OpenStreetMap
map = folium.Map(location=[41.842237, -93.248822], tiles="OpenStreetMap", zoom_start=9)

In [9]:
# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

In [10]:
# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    # assign a color marker for the type of volcano, Strato being the most common
    if geo_df.event_type[i] == "system malfunction":
        type_color = "green"
    elif geo_df.event_type[i] == "tank below 10%":
        type_color = "blue"
    elif geo_df.event_type[i] == "device error":
        type_color = "orange"
    elif geo_df.event_type[i] == "tank below 5%":
        type_color = "purple"
    else:
        type_color = "pink"

    # Place the markers with the popup labels and data
    map.add_child(
        folium.Marker(
            location=coordinates,
            popup="device_id: "
            + str(geo_df.device_id[i])
            + "<br>"
            + "manufacturer: "
            + str(geo_df.manufacturer[i])
            + "<br>"
            + "iot_signal_1: "
            + str(geo_df.iot_signal_1[i])
            + "<br>"
            + "iot_signal_2: "
            + str(geo_df.iot_signal_2[i]),
            icon=folium.Icon(color="%s" % type_color),
        )
    )
    i = i + 1


In [11]:
map